# Helpers

In [ ]:
!pip install pickle5

In [ ]:
!pip install keras_applications

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pickle5 as pickle
from pathlib import Path
import random
import tensorflow as tf
import keras
import string
from math import ceil
import random
import numpy as np
import sys
import os
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import mean_squared_error as mse

In [ ]:
def path_of(location):
    me_dir, me_file= os.path.split(os.path.abspath(__file__))
    return os.path.join(me_dir, location)

def load_pkl(filename):
    filename= filename
    data= None
    with open(filename, "rb") as handle:
        data= pickle.load(handle)
        handle.close()
    return data

def store_pkl(object, filename):
    filename= filename
    with open(filename, "wb") as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)
        handle.close()

def is_valid_file(filename):
    filename= path_of(filename)
    file= Path(filename)
    if file.is_file():
        return True
    return False


def to_same_shape(arr_of_items, required_shape):
    if len(arr_of_items) == 0:
        print("Error: tried to make shape: ", required_shape, ", but item is empty...")
        exit()
    if len(arr_of_items)>required_shape:
        return arr_of_items[:required_shape]
    res= []
    ind=0
    while len(res)<required_shape:
        res.append(arr_of_items[ind])
        ind= (ind+1)%len(arr_of_items)
    return res

def get_rand_str(size):
    return "".join(random.choice(string.ascii_uppercase + string.digits) for _ in range(size))

def save_model(keras_model, save_folder="./models", save_filename= None):
    if save_filename is None:
        me_dir, me_file= os.path.split(os.path.abspath(__file__))
        save_filename= me_file.split(".")[0]+".h5"
    op_file= path_of(save_folder +"/"+save_filename)
    keras_model.save(op_file)
    print("\n\n    Saved_model:", save_filename, "\n\n")

# Model

In [ ]:
def save(self, location= "./model_vid/"):
    self.model_cnn.save(location+"/model_cnn.h5")
    self.model_user.save(location+"/model_user.h5")
    self.model_full.save(location+"/model_full.h5")

# Train

In [ ]:
use_precomputed_dict_for_video= False


input_dimension = 88
audio_encode_dim = 88

# imageW, imageH= 112, 112

batch_size= 64

train_obj= load_pkl("/content/drive/MyDrive/Recommendation_system/train_v2.obj") #will load pkl data using func
test_obj= load_pkl("/content/drive/MyDrive/Recommendation_system/test_v2.obj")   #will load pkl data using func


user_enc= load_pkl("/content/drive/MyDrive/Recommendation_system/new_user_embedding_v2_510.obj")     #will load pkl data using func
user_size= len(user_enc.get(list(user_enc.keys())[0])) #user_size --> 915
#print(user_size)

audio = load_pkl("/content/drive/MyDrive/Recommendation_system/audio_embedding_510.obj")


def shuffle_single_epoch(ratings):  #will shuffle the provided data
    data_copied= ratings.copy() 
    random.shuffle(data_copied)
    return data_copied

    
def normalize_rate(rate):
    return rate/5

def de_normalize_rate(rate):
    return rate*5


def get_nth_batch(data):

    users, movies, nrates= [], [], [] # 3 empty list 
    
    for user_id, movie_id, rate in data:
        #user_id --> 11megha89 , movie_id --.tt014651 , rate --> 1,0,-1
        if user_enc.get(user_id) is None:    #Return user_vec --> 915
            continue
        try:
            m_vid= audio[movie_id]
            #print(m_vid.shape) -->(112, 112, 9)
        except:
            continue
        users.append(user_enc.get(user_id)) #Will 
        movies.append(m_vid)
        if int(rate) == -1:
               nrates.append([0,0,1])
        elif int(rate) == 1:
               nrates.append([0,1,0])
        elif int(rate) == 0:
               nrates.append([1,0,0])
               
    users= np.array(users, dtype=float)
    # print(users.shape) --> (64, 915)
    movies= np.array(movies, dtype=float)
    # print(movies.shape) --> (64, 112, 112, 9)
    nrates= np.array(nrates, dtype=float)
    #print(nrates.shape) --> (64,)

    assert len(users)==len(movies)==len(nrates)

    return users, movies, nrates



def test(model, test_data):
    test_acc,test_loss= [],[]
    test_final_acc,test_final_loss = [] , []
    batch_count= ceil(len(test_data)/batch_size)
    for batch_id in range(batch_count):
        print(" -> Testing Batch: ", batch_id)
        user, movie, rate= get_nth_batch(test_data, batch_id)
        loss , acc = model.evaluate([movie, user], rate)
        test_acc.append(acc)
        test_loss.append(loss)
        
    test_final_acc.append(sum(test_acc)/len(test_acc))
    test_final_loss.append(sum(test_loss)/len(test_acc))
    return sum(test_acc)/len(test_acc) , sum(test_loss)/len(test_acc)



def train(model, data, test_data= None, epochs=50):
    test_acc_main = 0
    batch_count= ceil(len(data)/batch_size)       
    for epoch_id in range(1, epochs+1):
        data= shuffle_single_epoch(data)
        print("\n\t---- Starting Epoch:", epoch_id, "----")
        
        for batch_id in range(batch_count):
            print(" -> Batch: ", batch_id)
            print(batch_id)
            user, movie, rate= get_nth_batch(data, batch_id,batch_size) 
            #print(rate.shape)                        
            model.fit([movie, user], rate, batch_size=batch_size, epochs=1)            
            
        if test_data is not None:
            test_acc , test_loss= test(model, test_data)
            print("TestAcc after Epoch",epoch_id,": ",test_acc)
            print("TestLoss after Epoch",epoch_id,": ",test_loss)
            if test_acc>test_acc_main:
                    test_acc_main = test_acc
                    location= "./model_vid/"
                    model.save(location+"/model_abc.h5")


In [ ]:
import pandas as pd
data_obj = test_obj + train_obj
pd_data = pd.DataFrame(data_obj)

results = []
for i in pd_data.iloc[:,2]:
    results.append(i[0])

pd_data[3] = results
x_data = np.array(pd_data.iloc[:,:2])
y_data = np.array(pd_data.iloc[:,3])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, stratify=y_data ,test_size=0.2)

train_obj = pd.DataFrame(X_train)
train_obj[3] = pd.DataFrame(y_train)
train_obj = np.array(train_obj)

test_obj = pd.DataFrame(X_test)
test_obj[3] = pd.DataFrame(y_test)
test_obj = np.array(test_obj)

In [ ]:
def audio_Main_Model(input_dim,audio_encode_dim,user_size):

    input_length  = input_dim
    print(input_length)
    audio_encode_dim = audio_encode_dim
    user_size = user_size
    
    #------------------------------------------AUDIO MODEL-------------------------------------------
    audio_input = keras.layers.Input(shape=(input_length),name="audio_input")
   
    #flatten
    dense1 = keras.layers.Dense(units = 88 , activation = "tanh")(audio_input)
    dense2 = keras.layers.Dense(units = 88 , activation = "tanh")(dense1)
    dense3 = keras.layers.Dense(units = 88 , activation = "tanh")(dense2)
    
    audio_model = keras.models.Model(inputs = audio_input , outputs = dense3)
    
    #-----------------------------------------USER MODEL------------------------------------------------------
    
    user1 = keras.layers.Input(shape=(user_size,) , name = "User_Input")
    user1_norm =  keras.layers.BatchNormalization()(user1)
    user_dense= keras.layers.Dense(units= int(user_size*2/3), activation="tanh")(user1_norm)
    
    user_model = keras.models.Model(inputs = user1 , outputs = user_dense)
        
    #-----------------------------------------Concatination--------------------------------------------------
    combined = keras.layers.concatenate([audio_model.output, user_model.output])
    combined_norm = keras.layers.BatchNormalization()(combined)
    concat_dense1 = keras.layers.Dense(units= int((audio_encode_dim + user_size)/2), activation="tanh")(combined_norm)
    concat_dense2 = keras.layers.Dense(units=1024, activation="sigmoid")(concat_dense1)    
    concat_dense3 = keras.layers.Dense(units=128, activation="sigmoid" )(concat_dense2)    
    concat_linear = keras.layers.Dense(units=1, activation="sigmoid")(concat_dense3)
    
    model = keras.models.Model(inputs=[audio_model.input, user_model.input], outputs = concat_linear)
    
    return model

In [ ]:
model = audio_Main_Model(input_dimension,audio_encode_dim,user_size)



In [ ]:
print(model.summary())

# Training 

In [ ]:
users_train, movies_train, nrates_train = get_nth_batch(train_obj)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'loss', patience = 12, mode = 'min', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics to the model.
model.compile(loss='mse', optimizer='RMSprop', metrics=['mean_squared_error'])

# model_training_history = model.fit(x = [movies_train,users_train], y = nrates_train, epochs = 100, batch_size = 16 , shuffle = True, validation_split=0.1) 
# Start training the model.
model_training_history = model.fit(x = [movies_train,users_train], y = nrates_train, epochs = 100, batch_size = 16 , shuffle = True, validation_split=0.1, callbacks = [early_stopping_callback])

# Testing

In [ ]:
users_test, movies_test, nrates_test = get_nth_batch(test_obj)

In [ ]:
model_evaluation_history = model.evaluate(x = [movies_test,users_test], y = nrates_test)

In [ ]:
model.summary()

# Embedding Extraction

In [ ]:
audio_features_model = tf.keras.models.Model(inputs=model.get_layer(name = "audio_input").input,
                      outputs=model.get_layer(name = "dense_2").output,)

In [ ]:
audio.keys()

In [ ]:
inputx = np.expand_dims(audio['tt0064086'], axis=0)
b = audio_features_model(inputx)

In [ ]:
l = list(audio.keys())
arrx = np.zeros([len(l),89])
arrx = pd.DataFrame(arrx)

In [ ]:
for i in range(len(l)):
    x = l[i]
    inputx = np.expand_dims(audio[x], axis=0)
    outputx = audio_features_model(inputx)
    y = np.array(outputx) 
    arrx.iloc[i,0] =  x
    arrx.iloc[i,1:] = y[0]

In [ ]:
arrx

In [ ]:
arrx.to_csv('audio_embeddings_flickscore.csv' , index=False)

In [ ]:
axx_np = np.array(arrx)
embeddings_dic = {}
for i in range(len(axx_np)):
    embeddings_dic[axx_np[i][0]] = list(axx_np[i][1:])

In [ ]:
import pickle 
file_pi = open('/content/drive/MyDrive/sony_IITPatna/dict_audio_embedding_fickscore.obj', 'wb') 
pickle.dump(embeddings_dic, file_pi)